# 目录
## 1. 导入模块
## 2. `tf.GradientTape`的基本用法
  - ### 2.1 初次使用
    - `tf.GradientTape`
	- `gradient`
	
  - ### 2.2 打开`persistent=True`
    - `tf.GradientTape(persistent=True)`
	- `gradient`
	
  - ### 2.3 参数一起求导
  - ### 2.4 常量必需要申明
    - `watch`
	
  - ### 2.5 两个函数对一个参数求导
  - ### 2.6 二次求导
  - ### 2.7 求凸函数的极值
    - `apply_gradients`
  
## 3. 应用：神经网络优化参数
  - ### 3.1 获取加利福尼亚数据
  - ### 3.2 封装的优化算法训练
  - ### 3.3 自定义优化算法训练
    - `keras.metrics.MeanSquaredError`
	- `reset_states`
	- `keras.optimizers.SGD`
	- `np.random.randint`
	- `tf.GradientTape`
	- `keras.losses.mean_squared_error`
	- `gradient`
	- `apply_gradients`

## 1. 导入模块

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn

from tensorflow import keras
import tensorflow as tf
import sys
import os
import time
import datetime

for module in [np, pd, mpl, sklearn, keras, tf]:
    print(module.__name__, module.__version__)

numpy 1.18.1
pandas 0.25.3
matplotlib 3.1.2
sklearn 0.22.1
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
tensorflow 2.1.0


## 2. `tf.GradientTape`的基本用法

In [2]:
def g(x1, x2):
    return (x1 + 5)*(x2**2)

  - ### 2.1 初次使用

In [3]:
v1 = tf.Variable(2.0)
v2 = tf.Variable(3.0)

with tf.GradientTape() as tape:
    z = g(v1, v2)

dz_v1 = tape.gradient(z, v1)
print(dz_v1)

try:
    dz_v2 = tape.gradient(z, v2) # tape 默认只能调用一次
    print(dz_v2)
except Exception as e:
    print(e)


tf.Tensor(9.0, shape=(), dtype=float32)
GradientTape.gradient can only be called once on non-persistent tapes.


  - ### 2.2 打开`persistent=True`

In [4]:
v1 = tf.Variable(2.0)
v2 = tf.Variable(3.0)

with tf.GradientTape(persistent=True) as tape: # persistent=True  tape 可以调用很多次
    z = g(v1, v2)

dz_v1 = tape.gradient(z, v1)
print(dz_v1)

dz_v2 = tape.gradient(z, v2)
print(dz_v2)

del tape

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(42.0, shape=(), dtype=float32)


  - ### 2.3 参数一起求导

In [5]:
v1 = tf.Variable(2.0)
v2 = tf.Variable(3.0)

with tf.GradientTape() as tape: 
    z = g(v1, v2)

dz_v1v2 = tape.gradient(z, [v1, v2]) # 参数可以一起求导
print(dz_v1v2)

[<tf.Tensor: shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: shape=(), dtype=float32, numpy=42.0>]


  - ### 2.4 常量必需要申明

In [6]:
v1 = tf.constant(2.0)
v2 = tf.constant(3.0)

with tf.GradientTape() as tape: 
    z = g(v1, v2)

dz_v1v2 = tape.gradient(z, [v1, v2]) 
print(dz_v1v2)

[None, None]


In [7]:
v1 = tf.constant(2.0)
v2 = tf.constant(3.0)

with tf.GradientTape() as tape: 
    tape.watch(v1)
    tape.watch(v2)
    z = g(v1, v2)

dz_v1v2 = tape.gradient(z, [v1, v2]) 
print(dz_v1v2)

[<tf.Tensor: shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: shape=(), dtype=float32, numpy=42.0>]


  - ### 2.5 两个函数对一个参数求导

In [8]:
x = tf.Variable(5.0)

with tf.GradientTape() as tape:
    z1 = x*3
    z2 = x**3

tape.gradient([z1, z2], x)

<tf.Tensor: shape=(), dtype=float32, numpy=77.99999>

  - ### 2.6 二次求导

In [9]:
v1 = tf.Variable(2.0)
v2 = tf.Variable(3.0)

with tf.GradientTape(persistent=True) as outer_tape:
    with tf.GradientTape(persistent=True) as inner_tape:
        z = g(v1, v2)
    inner_grads = inner_tape.gradient(z, [v1, v2])
outer_grads = [outer_tape.gradient(inner_grad, [v1, v2])for inner_grad in inner_grads]

del outer_tape
del inner_tape

print(outer_grads)

[[None, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>], [<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, <tf.Tensor: shape=(), dtype=float32, numpy=14.0>]]


  - ### 2.7 求凸函数的极值

In [10]:
def f(x):
    return 3. * x **2 + 2. * x - 1

#  初始化 x
x = tf.Variable(0.0)

learning_rate = 0.001

# 梯度下降
for _ in range(5000):
    with tf.GradientTape() as tape:
        y = f(x)
    dy_dx = tape.gradient(y, x)
    x.assign_sub(learning_rate * dy_dx)

print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-0.33333087>


In [11]:
def f(x):
    return 3. * x **2 + 2. * x - 1

#  初始化 x
x = tf.Variable(0.0)

learning_rate = 0.001

optimizer = keras.optimizers.SGD(lr=learning_rate)

# 梯度下降
for _ in range(5000):
    with tf.GradientTape() as tape:
        y = f(x)
    dy_dx = tape.gradient(y, x)
    optimizer.apply_gradients([(dy_dx, x)])

print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-0.33333087>


## 3. 应用：神经网络优化参数

  - ### 3.1 获取加利福尼亚数据

In [12]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state=12234)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state=1234)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

print(x_train_scaled.shape, y_train.shape)
print(x_valid_scaled.shape, y_valid.shape)
print(x_test_scaled.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


  - ### 3.2 封装的优化算法训练

In [16]:
model = keras.models.Sequential([
    keras.layers.Dense(100, input_shape=[8], activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(1e-3), metrics=["mse"])

history = model.fit(x_train_scaled, y_train, validation_data=(x_valid_scaled, y_valid), epochs=100, batch_size=32)

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 89us/sample - loss: 1.6360 - mse: 1.6360 - val_loss: 1.9553 - val_mse: 1.9553
Epoch 2/100
11610/11610 [==============================] - 1s 66us/sample - loss: 0.7432 - mse: 0.7432 - val_loss: 0.7036 - val_mse: 0.7036
Epoch 3/100
11610/11610 [==============================] - 1s 66us/sample - loss: 0.6359 - mse: 0.6359 - val_loss: 0.6331 - val_mse: 0.6331
Epoch 4/100
11610/11610 [==============================] - 1s 67us/sample - loss: 0.5844 - mse: 0.5844 - val_loss: 0.5750 - val_mse: 0.5750
Epoch 5/100
11610/11610 [==============================] - 1s 66us/sample - loss: 0.5485 - mse: 0.5485 - val_loss: 0.5451 - val_mse: 0.5451
Epoch 6/100
11610/11610 [==============================] - 1s 66us/sample - loss: 0.5214 - mse: 0.5214 - val_loss: 0.5154 - val_mse: 0.5154
Epoch 7/100
11610/11610 [==============================] - 1s 66us/sample - loss: 0.4992 - mse: 0.4992 - val_lo

Epoch 59/100
11610/11610 [==============================] - 1s 68us/sample - loss: 0.3384 - mse: 0.3384 - val_loss: 0.3428 - val_mse: 0.3428
Epoch 60/100
11610/11610 [==============================] - 1s 69us/sample - loss: 0.3373 - mse: 0.3373 - val_loss: 0.3400 - val_mse: 0.3400
Epoch 61/100
11610/11610 [==============================] - 1s 69us/sample - loss: 0.3364 - mse: 0.3364 - val_loss: 0.3406 - val_mse: 0.3406
Epoch 62/100
11610/11610 [==============================] - 1s 68us/sample - loss: 0.3359 - mse: 0.3359 - val_loss: 0.3463 - val_mse: 0.3463
Epoch 63/100
11610/11610 [==============================] - 1s 69us/sample - loss: 0.3355 - mse: 0.3355 - val_loss: 0.3399 - val_mse: 0.3399
Epoch 64/100
11610/11610 [==============================] - 1s 68us/sample - loss: 0.3347 - mse: 0.3347 - val_loss: 0.3349 - val_mse: 0.3349
Epoch 65/100
11610/11610 [==============================] - 1s 68us/sample - loss: 0.3343 - mse: 0.3343 - val_loss: 0.3518 - val_mse: 0.3518
Epoch 66/100


  - ### 3.3 自定义优化算法训练

小知识：metric 的用法是用于累加求度量

In [15]:
metric = keras.metrics.MeanSquaredError()

print(metric([2.0], [9.0]))
print(metric([1.0], [0.0]))
print(metric.result())

metric.reset_states()  # 重新开始
print(metric([2.0], [9.0]))


tf.Tensor(49.0, shape=(), dtype=float32)
tf.Tensor(25.0, shape=(), dtype=float32)
tf.Tensor(25.0, shape=(), dtype=float32)
tf.Tensor(49.0, shape=(), dtype=float32)


In [20]:
# 定义模型
model_gradient = keras.models.Sequential([
    keras.layers.Dense(100, input_shape=[8], activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1)
])

epochs = 100
batch_size = 32
steps_per_epoch = len(x_train) // batch_size

metric = keras.metrics.MeanSquaredError()
optimizer = keras.optimizers.SGD(1e-3)

def generate_batch(x, y, batch_size=32):
    idx = np.random.randint(0, len(x), size=batch_size)
    return x[idx], y[idx]

# 遍历epochs
for epoch in range(epochs):
    metric.reset_states()
    
    # 一个epoch遍历
    for step in range(steps_per_epoch):
        
        # 拿数据
        x_batch, y_batch = generate_batch(x_train_scaled, y_train)
        
        # 求梯度
        with tf.GradientTape() as tape:
            y_pred = model_gradient(x_batch)
            y_pred = tf.squeeze(y_pred, 1)
            loss = keras.losses.mean_squared_error(y_batch, y_pred)
            metric(y_batch, y_pred)
        grads = tape.gradient(loss, model_gradient.trainable_variables)
        
        # 更新参数
        grads_and_vars = zip(grads, model_gradient.trainable_variables)
        optimizer.apply_gradients(grads_and_vars)
        
        print("\r[train] epoch: {}, mse: {:.4f}".format(epoch, metric.result().numpy()), end="")
        
    # 验证
    y_valid_pred = model_gradient(x_valid_scaled)
    y_valid_pred = tf.squeeze(y_valid_pred, 1)
    valid_loss = keras.losses.mean_squared_error(y_valid_pred, y_valid)
    print("\t[valid] mes: {:.4f}".format(valid_loss))




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[train] epoch: 0, mse: 2.0710	[valid] mes: 2.2600
[train] epoch: 1, mse: 0.7963	[valid] mes: 0.8227
[train] epoch: 2, mse: 0.6594	[valid] mes: 0.9071
[train] epoch: 3, mse: 0.5974	[valid] mes: 0.9768
[train] epoch: 4, mse: 0.5948	[valid] mes: 0.8132
[train] epoch: 5, mse: 0.5529	[valid] mes: 0.5765
[train] epoch: 6, mse: 0.5161	[valid] mes: 0.5206
[train] epoch: 7, mse: 0.5092	[valid] mes: 0.4945
[train] epoch: 8, mse: 0.4716	[valid] mes: 0.4976
[train] epoch: 9, mse: 0.4598	[valid] mes: 0.5234
[train] 